In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
import warnings 
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')

In [ ]:
# loading dataset

In [ ]:
df=pd.read_csv("AIML Dataset.csv")

In [ ]:
df.head()

In [ ]:
df.info()


In [ ]:
df.columns

In [ ]:
df["isFraud"].value_counts()

In [ ]:
df["isFlaggedFraud"].value_counts()

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
#fraud percentage
fraud_percentage=(df["isFraud"].value_counts()[1]/df.shape[0])*100
print(fraud_percentage)

In [ ]:
df["type"].value_counts().plot(kind="bar",title="Transaction Type Distribution",color="lightblue")
plt.xlabel("Transaction Type")
plt.ylabel("Count")
plt.show() 

In [ ]:
fraud_by_types=df.groupby("type")["isFraud"].mean().sort_values(ascending=False)
fraud_by_types.plot(kind="bar",title="Fraud rate by Transaction Type",color="salmon")
plt.ylabel("Fraud Rate")

In [ ]:
df["amount"].describe().astype(int)

In [ ]:
sns.histplot(np.log1p(df["amount"]),bins=50,kde=True,color="purple")
plt.title("Transaction Amount Distribution(log scale)")
plt.xlabel("log(Amount+1)")
plt.show()

In [ ]:
sns.boxplot(data=df[df["amount"]<50000],x="isFraud",y="amount")
plt.title("boxplot of amount by fraud status")
plt.xlabel("isFraud")
plt.ylabel("Amount")
plt.show()

In [ ]:
df.columns

In [ ]:
df.drop(columns=["step"],inplace=True)

In [ ]:
df.head()

In [ ]:
top_sender=df["nameOrig"].value_counts().head(10)

In [ ]:
top_sender

In [ ]:
top_receiver=df["nameDest"].value_counts().head(10)

In [ ]:
top_receiver

In [ ]:
fraud_types=df[df["type"].isin(["TRANSFER","CASH_OUT"])]

In [ ]:
fraud_types["type"].value_counts()

In [ ]:
sns.countplot(data=fraud_types,x="type",hue="isFraud",palette="Set2")
plt.title("Fraud count by Transaction type CASH_OUT and TRANSFER")

In [ ]:
corr=df[[ "amount","oldbalanceOrg","newbalanceOrig","oldbalanceDest","newbalanceDest","isFraud"]].corr()

In [ ]:
corr

In [ ]:
sns.heatmap(corr,annot=True,cmap="coolwarm",fmt=".2f")
plt.title("Correlation matrix")
plt.show()

In [ ]:
zero_after_transfer=df[(df["oldbalanceOrg"]>0)&
                       (df["newbalanceOrig"]==0)&
                       (df["type"].isin(["TRANSFER","CASH_OUT"]))
                       ]
print("Number of transactions with zero new balance after transfer or cash out:",zero_after_transfer.shape)

In [ ]:
zero_after_transfer.head(10)

In [ ]:
df["isFraud"].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder


In [ ]:
df_model=df.drop(columns=["nameOrig","nameDest","isFlaggedFraud"])

In [ ]:
df_model.head()

In [ ]:
categorical=["type"]
numerical=["amount","oldbalanceOrg","newbalanceOrig","oldbalanceDest","newbalanceDest"]

In [ ]:
Y=df_model["isFraud"]
X=df_model.drop(columns=["isFraud"])

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state=42,stratify=Y)

In [ ]:
preprocessor=ColumnTransformer(transformers=[
    ("num",StandardScaler(),numerical),
    ("cat",OneHotEncoder(drop="first"),categorical)
    ],remainder="drop")


In [ ]:
pipeline=Pipeline([("prep",preprocessor),
                   ("clf",LogisticRegression(class_weight="balanced",max_iter=1000))

])

In [ ]:
pipeline.fit(X_train,Y_train)

In [ ]:
y_pred=pipeline.predict(X_test)

In [ ]:
print(classification_report(Y_test,y_pred))

In [ ]:
confusion_matrix(Y_test,y_pred)

In [ ]:
pipeline.score(X_test,Y_test)

In [ ]:
import joblib
joblib.dump(pipeline,"fraud_detection_model.pkl")

In [ ]:
X_test.head()

In [ ]:
X_train.columns


In [ ]:
X_train["type"].unique()